# Tp3 Part 2: Minimum-time Optimal Control (due Nov 27)

In [ ]:
import time
import numpy as np
from numpy.linalg import inv,norm,pinv,svd,eig
from scipy.linalg import solve_discrete_are
import matplotlib.pylab as plt
from copy import deepcopy
from tp3 import cartpole_loader
import pinocchio as pin
from utils.meshcat_viewer_wrapper import MeshcatVisualizer
from utils.load_ur5_with_obstacles import load_ur5
import pinocchio.casadi as cpin
import casadi as cs

If importing load_ur5 fails, you may need to install the example-robot-data package in the conda environment of Tp2 and Tp3. This can be done by
```
conda install example-robot-data=4.3.0 -c conda-forge
```

In [ ]:
# Load UR5 robot model
robot = load_ur5()
viz = MeshcatVisualizer(robot)
viz.display(robot.q0)

In [ ]:
viz.viewer.jupyter_cell()

In [ ]:
# Sample initial configuration and goal configuration
pin.seed(30)
q_initial = pin.randomConfiguration(robot.model)
viz.display(q_initial)

In [ ]:
q_sample = pin.randomConfiguration(robot.model)
q_goal = q_initial + 0.5 * (q_sample - q_initial)
viz.display(q_goal)

### Ex: 3.2.1: Implement an acceleration-bounded minimum time optimal control problem

In [ ]:
# Time-optimal point-to-point motion planning with joint acceleration inputs 

# Joint limits
q_min = robot.model.lowerPositionLimit
q_max = robot.model.upperPositionLimit
qdot_limit = robot.model.velocityLimit
q_acc_limit = 50 # joint acceleration limits (rad/s^2)

opti = cs.Opti()

N = 50
dt = opti.variable()  # time step (actually only almost time-optimal since dt is fixed for all time steps)
opti.set_initial(dt, 0.1)

q_traj = opti.variable(robot.model.nq, N+1)  # joint positions
qdot_traj = opti.variable(robot.model.nv, N+1)  #
qddot_control = opti.variable(robot.model.nv, N)  # joint accelerations (controls)

# Initial and final conditions
opti.subject_to(q_traj[:,0] == q_initial)
opti.subject_to(qdot_traj[:,0] == 0)
opti.subject_to(q_traj[:,-1] == q_goal)
opti.subject_to(qdot_traj[:,-1] == 0)

# Your code here
# Implement multiple-shooting Dynamics constraints (integration can be exact)
# Implement joint limit constraint, joint velocity limit constraint, joint acceleration limit constraint
# Minimize total time
# If you have trouble solving the problem, try to set initial guesses for the variables 

In [ ]:
opti.solver('ipopt', {'expand':True})
sol = opti.solve()
print("Optimal time:", sol.value(dt) * N)

In [ ]:
q_traj_opt = sol.value(q_traj)
v_traj_opt = sol.value(qdot_traj)
a_traj_opt = sol.value(qddot_control)
dt_opt = sol.value(dt)

In [ ]:
plt.figure()
plt.plot(np.arange(N+1)*dt_opt, q_traj_opt.T)
plt.title("Joint positions")
plt.xlabel("Time [s]")
plt.ylabel("Position [rad]")
plt.legend([f"q{i}" for i in range(robot.model.nq)])
plt.grid()
plt.figure()
plt.plot(np.arange(N+1)*dt_opt, v_traj_opt.T)
plt.title("Joint velocities")
plt.xlabel("Time [s]")
plt.ylabel("Velocity [rad/s]")
plt.grid()
plt.figure()
plt.plot(np.arange(N)*dt_opt, a_traj_opt.T)
plt.title("Joint accelerations")
plt.xlabel("Time [s]")
plt.ylabel("Acceleration [rad/s²]")
plt.grid()
plt.show()

In [ ]:
viz.viewer.jupyter_cell()

In [ ]:
for k in range(N+1):
    viz.display(q_traj_opt[:,k])
    time.sleep(dt_opt)

In [ ]:
# Get Casadi compatible model and data
cmodel = cpin.Model(robot.model) 
cdata = cmodel.createData()

q_sym = cs.SX.sym("q", robot.model.nq)
qdot_sym = cs.SX.sym("qdot", robot.model.nv)
qddot_sym = cs.SX.sym("qddot", robot.model.nv)
tau_sym = cpin.rnea(cmodel, cdata, q_sym, qdot_sym, qddot_sym)
rnea_fun = cs.Function("rnea_casadi", [q_sym, qdot_sym, qddot_sym], [tau_sym])

torque_limit = 20 #robot.model.effortLimit*0.5

tau_traj = []

# impose torque limits and store the torques in tau_traj
## Your code here

In [ ]:
sol = opti.solve()
q_traj_opt = sol.value(q_traj)
v_traj_opt = sol.value(qdot_traj)
a_traj_opt = sol.value(qddot_control)
tau_traj_opt = sol.value(cs.hcat(tau_traj))
dt_opt = sol.value(dt)
print("Optimal time with torque limits:", dt_opt * N)

In [ ]:
plt.figure()
plt.plot(np.arange(N+1)*dt_opt, q_traj_opt.T)
plt.title("Joint positions")
plt.xlabel("Time [s]")
plt.ylabel("Position [rad]")
plt.legend([f"q{i}" for i in range(robot.model.nq)])
plt.grid()
plt.show()  
plt.figure()
plt.plot(np.arange(N+1)*dt_opt, v_traj_opt.T)
plt.title("Joint velocities")
plt.xlabel("Time [s]")
plt.ylabel("Velocity [rad/s]")
plt.grid()
plt.figure()
plt.plot(np.arange(N)*dt_opt, a_traj_opt.T)
plt.title("Joint accelerations")
plt.xlabel("Time [s]")
plt.ylabel("Acceleration [rad/s²]")
plt.grid()
plt.figure()
plt.plot(np.arange(N)*dt_opt, tau_traj_opt.T)
plt.title("Joint torques")
plt.xlabel("Time [s]")
plt.ylabel("Torque [Nm]")
plt.grid()
plt.show()

Do the control signals look satisfactory? If not, explain why not and how it can be improved?